### Simple MD (3D) simulation

In [13]:
import numpy as np
import h5py
from polychrom.starting_conformations import grow_cubic
import matplotlib.pyplot as plt

In [37]:
##### Define parameters
trajectories = h5py.File("../1D_trajectory/trajectory/customExtruder_LEFPositions.h5") # Saved trajectories from 1D siumulation

N = trajectories.attrs["N"] # Length of polymer
LEFNum = trajectories.attrs["LEFNum"] # Number of extruders
LEFpositions = trajectories["positions"] # Positions of extruders at each 1D step
Nframes = LEFpositions.shape[0] # Number of 1D steps (= number of extruder steps)

steps = 100 # MD steps PER STEP OF EXTRUDER
stiff = 1
box = (N / 0.1) ** 0.33 # Cubed size of box in monomers
data = grow_cubic(N, int(box))

# SMC (Extruder) parameters
smcBondWiddleDist = 0.2
smcBondDist = 0.5

### Simulation saving parameters
saveEveryBlocks = 10 # Write coordinates every this many blocks
restartSimulationEveryBlocks = 100 # 
# Checks
assert Nframes % restartSimulationEveryBlocks == 0 # So we don't have leftover steps that won't get saved
assert (restartSimulationEveryBlocks % saveEveryBlocks) == 0

savesPerSim = restartSimulationEveryBlocks // saveEveryBlocks
simInitsTotal = Nframes // restartSimulationEveryBlocks # Number of simulation initializations

print("""
There will be {} MD steps done for every step of the extruder
Conformations saved every {} steps, for {} confs. per simulation
Simulation restarts every {} steps
      """.format(steps,saveEveryBlocks,savesPerSim,restartSimulationEveryBlocks))


There will be 100 MD steps done for every step of the extruder
Conformations saved every 10 steps, for 10 confs. per simulation
Simulation restarts every 100 steps
      
